In [8]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Lambda
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model
from keras.optimizers import Adam

import tensorflow as tf
import matplotlib.pyplot as plt

import numpy as np

import os

In [ ]:
def load_data(path):
    img_list_paths = os.listdi
    for 

In [18]:
#generator = deconv, NN, bilinear

class DCGAN():
    def __init__(self,generator='deconv'):
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 3

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator(generator)
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generates imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self,generator):

        noise_shape = (100,)

        model = Sequential()
        
        if generator == 'NN':
            
            model.add(Dense(16*16*1024, activation="relu", input_shape=noise_shape))
            model.add(Reshape((16, 16, 1024)))
            model.add(BatchNormalization(momentum=0.8))
            model.add(UpSampling2D())
            model.add(Conv2D(256, kernel_size=3, padding="same"))
            model.add(Activation("relu"))
            model.add(BatchNormalization(momentum=0.8))
            model.add(UpSampling2D())
            model.add(Conv2D(128, kernel_size=3, padding="same"))
            model.add(Activation("relu"))
            model.add(BatchNormalization(momentum=0.8))
            model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
            model.add(Activation("tanh"))

            model.summary()

            noise = Input(shape=noise_shape)
            img = model(noise)

            return Model(noise, img)
        
        elif generator == 'deconv':
            
            model.add(Dense(4*4*1024, activation="relu", input_shape=noise_shape))
            model.add(Reshape((4, 4, 1024)))
            
            model.add(BatchNormalization(momentum=0.8))
            model.add(Conv2DTranspose(filters = 512, kernel_size = 5, strides = 2,
                                      padding = 'same',
                                      activation='relu',
                                      kernel_initializer='glorot_normal'))
            
            model.add(BatchNormalization(momentum=0.8))
            model.add(Conv2DTranspose(filters = 256, kernel_size = 5, strides = 2,
                                      padding = 'same',
                                      activation='relu',
                                      kernel_initializer='glorot_normal'))
            
            model.add(BatchNormalization(momentum=0.8))
            model.add(Conv2DTranspose(filters = 128, kernel_size = 5, strides = 2,
                                      padding = 'same',
                                      activation='relu',
                                      kernel_initializer='glorot_normal'))
            
            model.add(BatchNormalization(momentum=0.8))
            model.add(Conv2DTranspose(filters = 3, kernel_size = 5, strides = 2,
                                      padding = 'same',
                                      activation='relu',
                                      kernel_initializer='glorot_normal'))
            
            model.summary()
            noise = Input(shape=noise_shape)
            img = model(noise)

            return Model(noise, img)
        
        elif generator == 'bilinear':
            model.add(Dense(16*16*1024, activation="relu", input_shape=noise_shape))
            model.add(Reshape((16, 16, 1024)))
            
            model.add(BatchNormalization(momentum=0.8))
            model.add(Lambda(lambda x: tf.image.resize_images(x,size=(32,32))))
            model.add(Conv2D(256, kernel_size=3, padding="same"))
            model.add(Activation("relu"))
            
            model.add(BatchNormalization(momentum=0.8))
            model.add(Lambda(lambda x: tf.image.resize_images(x,size=(64,64))))
            model.add(Conv2D(128, kernel_size=3, padding="same"))
            model.add(Activation("relu"))
            
            model.add(BatchNormalization(momentum=0.8))
            model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
            model.add(Activation("tanh"))

            model.summary()

            noise = Input(shape=noise_shape)
            img = model(noise)

            return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        #model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, X_train, epochs, batch_size=128):

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        
        
        

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            # Sample noise and generate a half batch of new images
            noise = np.random.normal(0, 1, (half_batch, 100))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

In [19]:
dcgan = DCGAN('deconv')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 16, 16, 64)        256       
__________

In [ ]:
#load the images
path = ''
X_train = load_data(path)

dcgan = DCGAN()
dcgan.train(X_train, epochs=4000, batch_size=32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 8, 8, 64)          0         
__________